In [1]:
import os
import datetime
from collections import namedtuple
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import base64

def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    creds = None
    token_file = f'token_{api_name}_{api_version}{prefix}.json'

    try:
        creds = Credentials.from_authorized_user_file(token_file, scopes[0])
    except FileNotFoundError:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, scopes[0])
        creds = flow.run_local_server(port=0)
        with open(token_file, 'w') as token:
            token.write(creds.to_json())

    return build(api_name, api_version, credentials=creds, static_discovery=False)

In [2]:
CLIENT_FILE = r"D:\reachinbox\python_reachinbox\client_secret_881370762830-o6d19v8ubh0pmjds77kvnf42qgl5s7i6.apps.googleusercontent.com.json"
API_NAME = "gmail"
API_VERSION = "v1"
SCOPES = ["https://mail.google.com/"]

In [3]:
service = create_service(CLIENT_FILE, API_NAME, API_VERSION, SCOPES)

In [4]:
unread_messages = service.users().messages().list(userId='me', q='is:unread').execute()

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("model")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\anshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
interested_reply = "Thank you for your interest in our product! Here are some details about it..."
not_interested_reply = "Thank you for letting us know. We appreciate your feedback."
prediction_label = {0: "Interested", 1: "Not Interested"}
to_email = 'anshushri0207@gmail.com'
subject = 'Test Email'

In [7]:
if 'messages' in unread_messages:
    for message in unread_messages['messages']:
        message_id = message['id']  # Get message ID
        message = service.users().messages().get(userId='me', id=message_id).execute()  # Fetch full message details
        
        sender_email, original_subject, message_id = None, None, None
        for header in message['payload']['headers']:
            if header['name'] == 'From':
                sender_email = header['value']
            elif header['name'] == 'Subject':
                original_subject = header['value']
            elif header['name'] == 'Message-ID':
                message_id = header['value']
            if sender_email and original_subject and message_id:
                break
        
        if not all([sender_email, original_subject, message_id]):
            print("Error: Sender email, original subject, or message ID not found")
            continue
        
        reply_subject = f"Re: {original_subject}"
        content = message['snippet']

        inputs = tokenizer(content, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
        print(f"Content: {content}")
        print(f"Prediction: {prediction_label[predicted_class]}")

        try:
            reply_body = interested_reply if predicted_class == 0 else not_interested_reply

            reply_message = f"From: {to_email}\r\nTo: {sender_email}\r\nSubject: {reply_subject}\r\nReferences: {message['threadId']}\r\nIn-Reply-To: {message_id}\r\n\r\n{reply_body}"

            create_message = {'raw': base64.urlsafe_b64encode(reply_message.encode()).decode(), 'threadId': message['threadId']}
            sent_message = service.users().messages().send(userId='me', body=create_message).execute()
            message_id = message['id']  # Get the ID of the sent message
            
            # Mark the original message as read
            service.users().messages().modify(userId='me', id=message_id, body={'removeLabelIds': ['UNREAD']}).execute()
            print("Email sent and marked as read!")
        except Exception as e:
            print(f"Error occurred while sending email: {str(e)}")
        print("-" * 50)


Content: gamil was granted access to your Google Account anshushri0711@gmail.com If you did not grant access, you should check this activity and secure your account. Check activity You can also see security
Prediction: Not Interested
Email sent and marked as read!
--------------------------------------------------
Content: Get Credit-smart, play the weekly Quiz Hi, Looks like you have been on top of your credit score game. Let&#39;s see if you take it up a notch with this week&#39;s quiz. Do you think getting married
Prediction: Interested
Email sent and marked as read!
--------------------------------------------------
Content: Download Shine app Mail by Ayush Engineering &amp; Fabrication Urgent Hiring For Software Developer, Exp: 0 - 0 Yrs Salary: 2.0 - 3.0 LPA Location: Gurugram,Delhi,Noida Apply now Dear Applicant, Thank
Prediction: Interested
Email sent and marked as read!
--------------------------------------------------
Content: Codash Solutions is hiring for Golang Developer 